In [1]:
import numpy as np
import pandas as pd

from transformer_thermal_model.cooler import CoolerType
from transformer_thermal_model.hot_spot_calibration import calibrate_hotspot_factor
from transformer_thermal_model.model import Model
from transformer_thermal_model.schemas import (
    InputProfile,
    ThreeWindingInputProfile,
    UserThreeWindingTransformerSpecifications,
    UserTransformerSpecifications,
    WindingSpecifications,
)
from transformer_thermal_model.transformer import PowerTransformer, ThreeWindingTransformer

one_week = 4 * 24 * 7
datetime_index = pd.date_range("2020-01-01", periods=one_week, freq="15min")

nominal_load = 100
load_points = pd.Series([nominal_load] * one_week, index=datetime_index)
ambient_temp = 21
temperature_points = pd.Series([ambient_temp] * one_week, index=datetime_index)

# Create an input object with the profiles
profile_input = InputProfile.create(
    datetime_index=datetime_index, load_profile=load_points, ambient_temperature_profile=temperature_points
)

# Initialise a power transformer with cooling type ONAF and, besides the mandatory user specifications, default values.
tr_specs = UserTransformerSpecifications(
    load_loss=1000,  # Transformer load loss [W]
    nom_load_sec_side=1500,  # Transformer nominal current secondary side [A]
    no_load_loss=200,  # Transformer no-load loss [W]
    amb_temp_surcharge=20,  # Ambient temperature surcharge [K]
)

transformer = PowerTransformer(user_specs=tr_specs, cooling_type=CoolerType.ONAF)

In [9]:

model = Model(temperature_profile=profile_input, transformer=transformer)
result = model.run()

In [26]:
one_week_steps = 4 * 24 * 7

continuous_load = pd.Series([transformer.specs.nom_load_sec_side] * one_week_steps, index=datetime_index)
ambient_temp_profile = pd.Series([ambient_temp] * one_week_steps, index=datetime_index)
model_input = InputProfile.create(
    datetime_index=datetime_index, load_profile=continuous_load, ambient_temperature_profile=ambient_temp_profile
)
uncalibrated_transformer = transformer
calibrated_trafo = calibrate_hotspot_factor(
    uncalibrated_transformer=uncalibrated_transformer,
    ambient_temp=20.0,
    hot_spot_limit=98,  # in most cases a hot-spot temperature limit of 98 can be used
    hot_spot_factor_min=1.1,
    hot_spot_factor_max=1.3,
)

In [14]:
# Define the time range for your simulation
datetime_index = [pd.to_datetime("2025-07-01 00:00:00") + pd.Timedelta(minutes=15 * i) for i in np.arange(0, 288)]

load_series_high = pd.Series(
    data=np.sin(np.arange(0, 288) * 900 * 2 * np.pi * 1 / 43200) * 500 + 500, index=datetime_index
)
load_series_middle = pd.Series(
    data=np.sin(np.arange(0, 288) * 900 * 2 * np.pi * 1 / 43200) * 500 + 500, index=datetime_index
)
load_series_low = pd.Series(
    data=np.sin(np.arange(0, 288) * 900 * 2 * np.pi * 1 / 43200) * 500 + 500, index=datetime_index
)

ambient_series = pd.Series(data=20, index=datetime_index)

# Create the input profile for the three-winding transformer
profile_input = ThreeWindingInputProfile.create(
    datetime_index=datetime_index,
    ambient_temperature_profile=ambient_series,
    load_profile_high_voltage_side=load_series_high,
    load_profile_middle_voltage_side=load_series_middle,
    load_profile_low_voltage_side=load_series_low,
)

# user_specs = UserThreeWindingTransformerSpecifications(
#     no_load_loss=20,
#     amb_temp_surcharge=10,
#     lv_winding=WindingSpecifications(
#         nom_load=1000, winding_oil_gradient=20, hot_spot_fac=1.2, time_const_winding=1, nom_power=1000
#     ),
#     mv_winding=WindingSpecifications(
#         nom_load=1000, winding_oil_gradient=20, hot_spot_fac=1.2, time_const_winding=1, nom_power=1000
#     ),
#     hv_winding=WindingSpecifications(
#         nom_load=2000, winding_oil_gradient=20, hot_spot_fac=1.2, time_const_winding=1, nom_power=2000
#     ),
#     load_loss_hv_lv=100,
#     load_loss_hv_mv=100,
#     load_loss_mv_lv=100,
# )
user_specs = UserThreeWindingTransformerSpecifications(
    no_load_loss=20,
    amb_temp_surcharge=10,
    lv_winding=WindingSpecifications(
        nom_load=1000, winding_oil_gradient=20,time_const_winding=1, nom_power=1000
    ),
    mv_winding=WindingSpecifications(
        nom_load=1000, winding_oil_gradient=20, time_const_winding=1, nom_power=1000
    ),
    hv_winding=WindingSpecifications(
        nom_load=2000, winding_oil_gradient=20, time_const_winding=1, nom_power=2000
    ),
    load_loss_hv_lv=100,
    load_loss_hv_mv=100,
    load_loss_mv_lv=100,
)
# Create the transformer object
transformer = ThreeWindingTransformer(user_specs=user_specs, cooling_type=CoolerType.ONAN)
model = Model(temperature_profile=profile_input, transformer=transformer)
results = model.run()

winding_oil_gradient=20.0 time_const_winding=1.0 hot_spot_fac=1.3 nom_load=1000.0 nom_power=1000.0


In [14]:
user_specs = UserThreeWindingTransformerSpecifications(
            no_load_loss=51000,
            amb_temp_surcharge=0,
            lv_winding=WindingSpecifications(
                nom_load=1560, winding_oil_gradient=15.0, time_const_winding=7, nom_power=30
            ),
            mv_winding=WindingSpecifications(
                nom_load=1100, winding_oil_gradient=15.5, time_const_winding=7, nom_power=100
            ),
            hv_winding=WindingSpecifications(
                nom_load=385, winding_oil_gradient=15.1, time_const_winding=7, nom_power=100
            ),
            load_loss_hv_lv=140500,
            load_loss_hv_mv=457300,
            load_loss_mv_lv=134000,
            )
uncalibrated_transformer = ThreeWindingTransformer(user_specs=user_specs, cooling_type=CoolerType.ONAN)

calibrated_trafo = calibrate_hotspot_factor(
    uncalibrated_transformer=uncalibrated_transformer,
    ambient_temp=20.0,
    hot_spot_limit=98,  # in most cases a hot-spot temperature limit of 98 can be used
    hot_spot_factor_min=1.1,
    hot_spot_factor_max=1.3,
)

In [15]:
calibrated_trafo.specs.hv_winding.hot_spot_fac

np.float64(1.16)